<a href="https://colab.research.google.com/github/swetha4444/Artificial-Intelligence/blob/master/183_assign6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Assignment 6

Davis–Putnam–Logemann–Loveland (DPLL) algorithm
Implement David-Putnam algorithm for checking the satisfiability of a sentence represented in propositional logic.

```
Swetha Saseendran
CSE-C
185001183
```




In [167]:
class DPLL:

    true_literals = set()
    false_literals = set()
    up,pr,branches = 0,0,0
    clauses = []
    literals = []



    def __init__(self, formula):
      #extracting all literals from all clauses
      self.literals = [literal for literal in list(set(formula)) if literal.isalpha()]
      #extracting all clauses from the formula
      self.clauses = formula.split(", ")
      if (len(self.literals) == 0):
        self.clauses = []
      print("INPUT CNF: ",self)

    def __str__(self):
        """To convert the Pythonic-list used for CNF notation into a readable formula in CNF. """
        
        cnf_str = ""
        
        for clause in self.clauses:  #represent each claue in CNF
            if len(clause) > 0:
                cnf_str += '(' + clause.replace(' ', ' ∨ ') + ') ∧ '
    
        if cnf_str == "":       #formula is empty
            cnf_str = "()"
            
        if cnf_str[-2] == "∧":  #removing the final '∧' added which is un-wanted.
            cnf_str = cnf_str[:-2:] 
    
        return cnf_str
        

    def find_literals(self, formula):
        literals = [literal for literal in list(set(''.join(formula))) if literal.isalpha()]
        return literals

    def remove_pure_lit(self):
        clauses = list(set(self.clauses))
        unit_clauses = [clause for clause in clauses if len(clause) < 3]
        unit_clauses = list(set(unit_clauses))
        current_literals = self.find_literals(clauses)  #Find literals in the current formula
        pure_literals = set()
        self.pr += 1
        for literal in current_literals:  #Check for literals like X
            is_pure = False
            for i in range(len(clauses)):
                if '¬' + literal not in clauses[i]: 
                    is_pure = True
                else:  #If ¬X exists in other clauses
                    is_pure = False
                    break
            if is_pure:  #If X is a pure literal
                pure_literals.add(literal)
                self.true_literals.add(literal)  #Add X to True literals
                i = 0
                while True:  #Remove clauses containing X
                    if i >= len(clauses):
                        break
                    if literal in clauses[i]:
                        clauses.remove(clauses[i])
                        i -= 1
                    i += 1

        for literal in current_literals:  #Check for literals like ¬X 
            literal = '¬' + literal  #Search for literals of the type ¬X
            is_pure = False
            for i in range(len(clauses)):
                if literal in clauses[i]: 
                    is_pure = True
                else:  #If X exists in other clauses
                    is_pure = False
                    break
            
            if is_pure:  #If ¬X is a pure literal
                pure_literals.add(literal)
                self.false_literals.add(literal[-1])  #Add X to False literals
                i = 0
                while True:  #Remove clauses containing ¬X
                    if i >= len(clauses):
                        break
                    if literal in clauses[i]:
                        clauses.remove(clauses[i]) 
                        i -= 1
                    i += 1
        self.clauses = clauses
        print("Pure Literals:\t\t\t\t", list(pure_literals))
        print("CNF after removing Pure Literals:\t", self)
        return clauses



    def unit_prop(self):
      print("\nCurrent Formula:\t\t", self)  
      new_true_literals  =  []
      new_false_literals  =  []    
       
      clauses = list(set(self.clauses)) #unique clauses
      unit_clauses = [clause for clause in clauses if len(clause) < 3] #implies X or ¬X
      unit_clauses = list(set(unit_clauses)) #unique unit clauses
      print("CNF before Unit Propagation:\t", self)

      if unit_clauses:
        for unit_clause in unit_clauses:
          self.up += 1
          if '¬' in unit_clause: #¬X found
            #assign False to it
            self.false_literals.add(unit_clause[-1]) #adding literal
            new_false_literals.append(unit_clause[-1])
            i = 0
            while True:
              #unit subsumption
              if unit_clause in clauses[i]:  #Remove the unit clause from the formula
                clauses.remove(clauses[i])   #Entire clause becomes True hence no sig in entire cnf
                i -= 1
              #unit resolution
              elif unit_clause[-1] in clauses[i]:  #If a ¬X exists in the formula
                clauses[i] = clauses[i].replace(unit_clause[-1], '').strip() #V False => no sig in clause
              i += 1
              if (i >= len(clauses)):
                break
          else:  #X found
            self.true_literals.add(unit_clause)
            new_true_literals.append(unit_clause)
            i = 0
            while True:
              #unit resolution
              if (('¬' + unit_clause) in clauses[i]):  #If a ¬X exists in the formula
                clauses[i] = clauses[i].replace('¬' + unit_clause, '').strip() #V False => no sig in clause
                if ('  ' in clauses[i]):  #Empty clause
                  clauses[i] = clauses[i].replace('  ', ' ')
              #unit subsumption
              elif unit_clause in clauses[i]:  #Remove the unit clause from the formula
                clauses.remove(clauses[i]) #Entire clause becomes True hence no sig in entire cnf
                i -= 1
              i += 1
              if (i >= len(clauses)):
                break
      self.clauses = clauses 
      print("CNF after Unit Propagation:\t", self)
      return new_true_literals,new_false_literals,self.clauses


    def DPLL_solver(self):
      self.branches += 1
      self.true_literals = set(self.true_literals)
      self.false_literals = set(self.false_literals)

      new_true_literals  =  []
      new_false_literals  =  []

      #Unit propogation
      new_true_literals,new_false_literals,clauses = self.unit_prop()
      
      if len(self.clauses) == 0:  #Formula becomes an empty clause
        return True
      
      if sum(len(clause) == 0 for clause in clauses):  #If there are empty parantheses, indicating contradiction
        for literal in new_true_literals:   #clear every newly set true literal
          self.true_literals.remove(literal)
        for literal in new_false_literals:  #clear every newly set false literal
          self.false_literals.remove(literal)
        print("\nNull clause found. \n\tBacktracking...")
        return False  #the formula is unsatisfiable.

      #Pure Literals
      clauses = self.remove_pure_lit()

      if len(self.clauses) == 0:  #Formula becomes an empty clause
        return True

      self.literals = self.find_literals(self.clauses) #form the literals again from the updated formula
      first_literal = self.literals[0]  #choose the first literal to set as True/False to find satisfying assignments
      
      #Branch First
      print("\nTrying with {0} as True:-".format(first_literal))
      self.clauses  = clauses + [first_literal]  #Try with first literal set to True => just append the literal as a clause   
      if self.DPLL_solver():  #recursively work on the new formula
        return True
      
      #Branch Second
      self.clauses = clauses + ['¬' + first_literal]  #Try with first literal set to False (i.e ¬(first literal) is True)
      print("\nTrying with ¬{0} as True:-".format(first_literal))
      if self.DPLL_solver():  #recursively work on the new formula
        return True

      else:  #no satisfying assignments were found to the literals of the formula
        self.clauses = clauses
        for literal in new_true_literals:   #remove all the assignments
          self.true_literals.remove(literal)
        for literal in new_true_literals:   #remove all the assignments
          self.true_literals.remove(literal)
        return False  #the formula is unsatisfiable.


    def print_result(self, satisfiability):
        print("-----------------------------------------")
        if satisfiability == True:
            print("The given CNF statement is satisfiable.")
            print("Assignments: ")
            for literal in self.true_literals:
                print("\t"+literal, "= True") 
            for literal in self.false_literals:
                print("\t"+literal, "= False") 

            assign_lit = list(self.true_literals) + list(self.false_literals)
            extra =  [i for i in self.literals +  assign_lit  if i in self.literals and i not in  assign_lit]
            for literal in extra:
              print("\t"+literal, "= True/False") 
        else:
            print("\nThe given CNF statement was unsatisfiable.") 

        print("\nNo. of branches:\t\t", self.branches)
        print("\nNo. of unit propagations:\t", self.up)
        print("\nNo. of pure literals recursions:", self.pr)
        print("-----------------------------------------\n\n")
        

In [168]:
solver = DPLL("X Y Z, X ¬Y, ¬X Y ¬Z, ¬Z")
sat = solver.DPLL_solver()
solver.print_result(sat)


INPUT CNF:  (X ∨ Y ∨ Z) ∧ (X ∨ ¬Y) ∧ (¬X ∨ Y ∨ ¬Z) ∧ (¬Z) 

Current Formula:		 (X ∨ Y ∨ Z) ∧ (X ∨ ¬Y) ∧ (¬X ∨ Y ∨ ¬Z) ∧ (¬Z) 
CNF before Unit Propagation:	 (X ∨ Y ∨ Z) ∧ (X ∨ ¬Y) ∧ (¬X ∨ Y ∨ ¬Z) ∧ (¬Z) 
CNF after Unit Propagation:	 (X ∨ ¬Y) ∧ (X ∨ Y) 
Pure Literals:				 ['X']
CNF after removing Pure Literals:	 ()
-----------------------------------------
The given CNF statement is satisfiable.
Assignments: 
	X = True
	Z = False
	Y = True/False

No. of branches:		 1

No. of unit propagations:	 1

No. of pure literals recursions: 1
-----------------------------------------




In [169]:
solver = DPLL("X Y, Y ¬X, ¬X ¬Y")
sat = solver.DPLL_solver()
solver.print_result(sat)


INPUT CNF:  (X ∨ Y) ∧ (Y ∨ ¬X) ∧ (¬X ∨ ¬Y) 

Current Formula:		 (X ∨ Y) ∧ (Y ∨ ¬X) ∧ (¬X ∨ ¬Y) 
CNF before Unit Propagation:	 (X ∨ Y) ∧ (Y ∨ ¬X) ∧ (¬X ∨ ¬Y) 
CNF after Unit Propagation:	 (X ∨ Y) ∧ (¬X ∨ ¬Y) ∧ (Y ∨ ¬X) 
Pure Literals:				 []
CNF after removing Pure Literals:	 (X ∨ Y) ∧ (Y ∨ ¬X) ∧ (¬X ∨ ¬Y) 

Trying with Y as True:-

Current Formula:		 (X ∨ Y) ∧ (Y ∨ ¬X) ∧ (¬X ∨ ¬Y) ∧ (Y) 
CNF before Unit Propagation:	 (X ∨ Y) ∧ (Y ∨ ¬X) ∧ (¬X ∨ ¬Y) ∧ (Y) 
CNF after Unit Propagation:	 (¬X) 
Pure Literals:				 ['¬X']
CNF after removing Pure Literals:	 ()
-----------------------------------------
The given CNF statement is satisfiable.
Assignments: 
	Y = True
	X = False

No. of branches:		 2

No. of unit propagations:	 1

No. of pure literals recursions: 2
-----------------------------------------




In [170]:
solver = DPLL("X, ¬X, Y, ¬Y")
sat = solver.DPLL_solver()
solver.print_result(sat)



INPUT CNF:  (X) ∧ (¬X) ∧ (Y) ∧ (¬Y) 

Current Formula:		 (X) ∧ (¬X) ∧ (Y) ∧ (¬Y) 
CNF before Unit Propagation:	 (X) ∧ (¬X) ∧ (Y) ∧ (¬Y) 
CNF after Unit Propagation:	 ()

Null clause found. 
	Backtracking...
-----------------------------------------

The given CNF statement was unsatisfiable.

No. of branches:		 1

No. of unit propagations:	 4

No. of pure literals recursions: 0
-----------------------------------------




In [171]:
solver = DPLL("P, ¬P")
sat = solver.DPLL_solver()
solver.print_result(sat)

INPUT CNF:  (P) ∧ (¬P) 

Current Formula:		 (P) ∧ (¬P) 
CNF before Unit Propagation:	 (P) ∧ (¬P) 
CNF after Unit Propagation:	 ()

Null clause found. 
	Backtracking...
-----------------------------------------

The given CNF statement was unsatisfiable.

No. of branches:		 1

No. of unit propagations:	 2

No. of pure literals recursions: 0
-----------------------------------------


